In [1]:
import pandas as pd
import pickle 
import xml.etree.cElementTree as ET
from bs4 import BeautifulSoup
import requests
import urllib
import os
import zipfile
import lxml
from lxml import etree

# 1. Data Download


In [6]:
import patent_extraction_utils as peu

In [ ]:
baselink = 'https://bulkdata.uspto.gov/data/patent/grant/redbook/bibliographic/'
for year in range(1991, 2019, 1):
    link = baselink + str(year) + '/'
    target_dir = '../data/grants_bibliography/'+str(year)+'/'
    peu.download_all_links(link, target_dir)

In [ ]:
download_all_links(lnk, )

In [4]:
def download_applications(year):
    headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:57.0) Gecko"\
               "/20100101 Firefox/57.0",}

    lnk = 'https://bulkdata.uspto.gov/data/patent/application/redbook/fulltext/'\
          + str(year) + '/'
    rr = requests.get(lnk, headers=headers)
    html = rr.content
    soup = BeautifulSoup(html, "html.parser")
    urls = [x.get('href') for x in soup.find_all('a') 
            if '.zip' in x.get('href')]
    target_dir = 'data/apps/' + str(year) + '/'
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)
    [urllib.request.urlretrieve(lnk + x, 
                                'data/apps/' + str(year) + '/'+x) for
     x in urls]

In [4]:
year = 2017
#download_applications(year)
olddir = os.getcwd()
newdir = '/work/data/apps/' + str(year) + '/'
os.chdir(newdir)

# 2. Split Files

In [ ]:
def unzip_file(fnm):
    zip_ref = zipfile.ZipFile(fnm, 'r')
    zip_ref.extractall('.')
    zip_ref.close()
    return fnm.split('/')[-1].replace('.zip', '.xml')

def split_files():
    for fnm in [x for x in os.listdir() if x.split('.')[-1]=='zip']:
        fnm_xml = unzip_file(fnm)
        print(fnm_xml)
        !bash /work/split_apps.sh

In [3]:
 {fnm_xml}
        !rm {fnm}

# 3. Meta Data Extraction

In [4]:
def extract_singles_from_xpath(tree, xpath):
    fsub = tree.xpath(xpath)
    if fsub == []:
        return ''
    return ''.join(fsub[0].itertext())

def extract_dict_from_xpath(tree, xpath):
    fsub = tree.xpath(xpath)
    if fsub == []:
        return {}
    ref_dict = {x.tag: x.text for x in fsub[0].iterchildren()}
    ref_dict.update({'len':len(fsub)})
    return ref_dict

def extract_all_singles_from_xpath(tree, xpaths):
    out = {v:extract_singles_from_xpath(tree, k) for k, v in xpaths.items()}
    return out
    
def extract_all_xpath_dicts(tree, xpaths):
    keys = []
    values = []
    
    for k, v in xpaths.items():
        edict = extract_dict_from_xpath(tree, k)
        keys.extend([v + '_' + x for x in edict.keys()])
        values.extend(list(edict.values()))
    
    edict = dict(zip(keys, values))
    return edict

In [5]:
def get_details_from_xml(fnm):
    tree = etree.parse(fnm)
    xpaths_singles = {
        '/us-patent-application/us-bibliographic-data-application/assignees/'+\
        'assignee/addressbook/orgname': 'assignee',
        '/us-patent-application/us-bibliographic-data-application/'+\
        'invention-title': 'title',
        '/us-patent-application/us-bibliographic-data-application/'+\
        'us-parties/us-applicants/us-applicant/addressbook/orgname': 
        'us-applicant'
    }
    
    xpaths_dicts = {
        '/us-patent-application/us-bibliographic-data-application/'+\
        'application-reference/document-id': 'app',
        '/us-patent-application/us-bibliographic-data-application/'+\
        'publication-reference/document-id': 'pub'
    }
    out_dict = extract_all_singles_from_xpath(tree, xpaths_singles)
    out_dict.update(extract_all_xpath_dicts(tree, xpaths_dicts))
    out_dict.update({'fnm':fnm})
    return out_dict

def get_details_from_folder(file_dir):
    dict_list = [get_details_from_xml(file_dir + x) for x in 
                                     os.listdir(file_dir)]
    df = pd.DataFrame.from_dict(dict_list)
    return df

In [6]:
def get_details_from_all_folders(basedir, year):
    df = [get_details_from_folder(basedir + x + '/') for x in 
           os.listdir(basedir)]
    df = pd.concat(df)
    df.to_csv('/work/data/apps/' + str(year) + '_all_apps.csv')
    return df
    

In [7]:
def make_year(year):
    os.chdir('/work/')
    download_applications(year)
    olddir = os.getcwd()
    newdir = '/work/data/apps/' + str(year) + '/'
    print(newdir)
    os.chdir(newdir)
    split_files()
    get_details_from_all_folders(newdir, year)
    return meta_df

In [8]:
def dsplit_year(year):
    os.chdir('/work/')
    download_applications(year)
    olddir = os.getcwd()
    newdir = '/work/data/apps/' + str(year) + '/'
    print(newdir)
    os.chdir(newdir)
    #split_files()

In [10]:
list(reversed(list(range(2006, 2019, 1))))

[2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006]

In [ ]:
[dsplit_year(x) for x in  list(reversed(list(range(2005, 2013, 1))))]

/work/data/apps/2012/
/work/data/apps/2011/
/work/data/apps/2010/
/work/data/apps/2009/


In [ ]:
%%time
outdfs = [make_year(x) for x in  list(reversed(list(range(2005, 2019, 1))))]

In [14]:
list(reversed(list(range(2005, 2016, 1))))

[2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005]

# 3. Looking at the doc-numbers

In [ ]:
from lxml import etree

In [ ]:
def extract_singles_from_xpath(tree, xpath):
    fsub = tree.xpath(xpath)
    if fsub == []:
        return ''
    return ''.join(fsub[0].itertext())

def extract_dict_from_xpath(tree, xpath):
    fsub = tree.xpath(xpath)
    if fsub == []:
        return {}
    ref_dict = {x.tag: x.text for x in fsub[0].iterchildren()}
    ref_dict.update({'len':len(fsub)})
    return ref_dict

def extract_all_singles_from_xpath(tree, xpaths):
    out = {v:extract_singles_from_xpath(tree, k) for k, v in xpaths.items()}
    return out
    
def extract_all_xpath_dicts(tree, xpaths):
    keys = []
    values = []
    
    for k, v in xpaths.items():
        edict = extract_dict_from_xpath(tree, k)
        keys.extend([v + '_' + x for x in edict.keys()])
        values.extend(list(edict.values()))
    
    edict = dict(zip(keys, values))
    return edict

In [ ]:
def get_details_from_xml(fnm):
    tree = etree.parse(fnm)
    xpaths_singles = {
        '/us-patent-application/us-bibliographic-data-application/assignees/'+\
        'assignee/addressbook/orgname': 'assignee',
        '/us-patent-application/us-bibliographic-data-application/'+\
        'invention-title': 'title',
        '/us-patent-application/us-bibliographic-data-application/'+\
        'us-parties/us-applicants/us-applicant/addressbook/orgname': 
        'us-applicant'
    }
    
    xpaths_dicts = {
        '/us-patent-application/us-bibliographic-data-application/'+\
        'application-reference/document-id': 'app',
        '/us-patent-application/us-bibliographic-data-application/'+\
        'publication-reference/document-id': 'pub'
    }
    out_dict = extract_all_singles_from_xpath(tree, xpaths_singles)
    out_dict.update(extract_all_xpath_dicts(tree, xpaths_dicts))
    out_dict.update({'fnm':fnm})
    return out_dict
    

In [ ]:
from tqdm import tqdm

In [ ]:
def get_details_from_folder(file_dir):
    #from IPython.core.debugger import Tracer; Tracer()()
    dict_list = [get_details_from_xml(file_dir + x) for x in 
                                     os.listdir(file_dir)]
    df = pd.DataFrame.from_dict(dict_list)
    return df

In [ ]:
%%time
all_df = get_details_from_folder('/work/data/apps/2017/ipa170202/')

In [ ]:
basedir = '/work/data/apps/2017/'
dfs = [get_details_from_folder(basedir + x + '/') for x in os.listdir(basedir)]
pd.concat(dfs).to_csv('/work/data/apps/2017_all_apps.csv')

In [3]:
all_apps = pd.read_csv('/work/data/apps/2017_all_apps.csv', low_memory=False).iloc[:, 1:]
all_apps.head()

,app_country,app_date,app_doc-number,app_len,assignee,fnm,pub_country,pub_date,pub_doc-number,pub_kind,pub_len,title,us-applicant
0,US,20151203,15540631,1,"Bridgestone Americas Tire Operations, LLC",/work/data/apps/2017/ipa171221/F_1722.xml,US,20171221,20170361662,A1,1,RADAR WEAR SENSING FOR TIRE APPLICATIONS,"Bridgestone Americas Tire Operations, LLC"
1,US,20151204,15532203,1,NaN,/work/data/apps/2017/ipa171221/F_959.xml,US,20171221,20170360899,A1,1,TREATING SEIZURE WITH RECOMBINANT ALKALINE PHO...,"Alexion Pharmaceuticals, Inc."
2,US,20151203,15532255,1,NaN,/work/data/apps/2017/ipa171221/F_1012.xml,US,20171221,20170360952,A1,1,BIOMOLECULE CONJUGATES,CELGENE CORPORATION
3,US,20151203,15533313,1,NaN,/work/data/apps/2017/ipa171221/F_2534.xml,US,20171221,20170362474,A1,1,Bonding Method and Conveyor Belt,"The Yokohama Rubber Co., Ltd."
4,US,20151207,15533160,1,NaN,/work/data/apps/2017/ipa171221/F_1033.xml,US,20171221,20170360973,A1,1,ELECTROCHEMICAL DEVICE FOR RELEASING IONS,NMR Technology AS


In [ ]:
%%time
for y in list(reversed(list(range(2006, 2019, 1)))):
    newdir = '/work/data/apps/' + str(y) + '/'
    os.chdir(newdir)
    get_details_from_all_folders(newdir, y)